In [30]:
import neurokit2 as nk
import glob
import mne
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import numpy as np
%matplotlib qt


In [27]:
paths1 = glob.glob('../../data/mne_raw/sub?-day?-*-raw.fif')
paths1 = [path for path in paths1 if 'mmn-raw' not in path]
paths1.sort()
paths2 = glob.glob('../../data/mne_raw/sub*-control-mindfulness-raw.fif')
paths2.sort()
paths = paths1 + paths2

In [35]:
df_all = pd.DataFrame()
verbose = False 

for path in paths:
    
    # get component names
    components = re.split('[-_]', os.path.basename(path))
    sub = components[0][3:]
    day = components[1]
    condition = components[2]        
    if day == "control":
        sub = int(sub)+10
        condition = "control"
    

    raw = mne.io.read_raw(path, preload=True)
    data = raw.get_data()
    ecg_index = raw.ch_names.index("ecg_heart")
    ecg = data[ecg_index]*-1
    
    eda_index = raw.ch_names.index("skin")
    eda = data[eda_index]
    
    rsp_index = raw.ch_names.index("breath")
    rsp = data[rsp_index]
    
    # Visualise biosignals
    if verbose:
        data = pd.DataFrame({"ECG": ecg,
                             "RSP": rsp,
                             "EDA": eda})
        nk.signal_plot(data, subplots=True)
        
    
    #########################
    ### EDA ###
    #########################
    
    signals, info = nk.eda_process(eda, sampling_rate=256)
    #nk.eda_plot(signals)
    
    scl = signals['EDA_Tonic']
    mean_scl = scl.mean()
    scr_peaks = signals['SCR_Peaks']
    scr_amplitude = signals['SCR_Amplitude']
    
    #extracted values
    mean_scl = scl.mean()
    num_scrs = len(scr_peaks)
    mean_scr_amplitude = scr_amplitude.mean()
    
    #########################
    ### ECG ###
    #########################
    
    signals, info = nk.ecg_process(ecg, sampling_rate=256)
    #nk.ecg_plot(signals)

    plot = nk.ecg_plot(signals)
    plt.savefig(f'figures/ekg_figures/{path[19:-4]}.png')
    plt.close()
    
    heart_rate = signals['ECG_Rate']
    hrv_time = nk.hrv_time(signals, sampling_rate=256)
    hrv_freq = nk.hrv_frequency(signals, sampling_rate=256)
    
    # Extract specific values
    mean_hr = heart_rate.mean()
    sdnn = hrv_time['HRV_SDNN'].iloc[0]
    rmssd = hrv_time['HRV_RMSSD'].iloc[0]
    lf = hrv_freq['HRV_LF'].iloc[0]
    hf = hrv_freq['HRV_HF'].iloc[0]
    lf_hf_ratio = hrv_freq['HRV_LFHF'].iloc[0]
    
    #########################
    ### RSP ###
    #########################
    
    signals, info = nk.rsp_process(rsp, sampling_rate=256)
    #nk.rsp_plot(signals)
    
    # extracting values
    respiratory_rate = signals['RSP_Rate']
    respiratory_amplitude = signals['RSP_Amplitude']
    respiratory_cycle_duration = 60 / respiratory_rate
    
    # Compute mean values
    mean_rr = respiratory_rate.mean()
    mean_amplitude = respiratory_amplitude.mean()
    mean_cycle_duration = respiratory_cycle_duration.mean()
    
    # Compute variability (standard deviation)
    std_rr = respiratory_rate.std()
    std_amplitude = respiratory_amplitude.std()
    std_cycle_duration = respiratory_cycle_duration.std()
    
    
    data = {
        "sub": sub,
        "day": day,
        "condition": condition, 
        "button": "all",
        "jhana": "all",
        "ch_name": "",
        "n_epochs": "",
        "n_channels": "",

        "Mean_SCL": [np.round(mean_scl,3)],
        "Num_SCRs": [num_scrs,],
        "Mean_SCR_Amplitude": [np.round(mean_scr_amplitude*10**6,3)],
        "Mean_HR": [np.round(mean_hr,3)],
        "SDNN": [np.round(sdnn,3)],
        "RMSSD": [np.round(rmssd,3)],    
        "LF": [np.round(lf*10**3,3)],
        "HF": [np.round(hf*10**3,3)],
        "LF_HF_Ratio": [np.round(lf_hf_ratio,3)],
        "Mean_RR": [np.round(mean_rr,3)],
        "Mean_Amplitude": [np.round(mean_amplitude,3)],
        "Mean_Cycle_Duration": [np.round(mean_cycle_duration,3)],
        "RR_Variability": [np.round(std_rr,3)],
        "Amplitude_Variability": [np.round(std_amplitude,3)],
        "Cycle_Duration_Variability": [np.round(std_cycle_duration,3)]
    }
    
    df_new = pd.DataFrame(data)
    df_all = pd.concat([df_all, df_new], ignore_index=True)
    df_all.to_csv('bio_data_new.csv', index=False)
    

Opening raw data file ../../data/mne_raw/sub0-day1-jhana-raw.fif...
    Range : 256 ... 388359 =      1.000 ...  1517.027 secs
Ready.
Reading 0 ... 388103  =      0.000 ...  1516.027 secs...


/opt/anaconda3/envs/hypyp/lib/python3.8/site-packages/neurokit2/ecg/ecg_plot.py:70: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


Opening raw data file ../../data/mne_raw/sub0-day1-mindfulness-raw.fif...
    Range : 256 ... 385039 =      1.000 ...  1504.059 secs
Ready.
Reading 0 ... 384783  =      0.000 ...  1503.059 secs...


/opt/anaconda3/envs/hypyp/lib/python3.8/site-packages/neurokit2/ecg/ecg_plot.py:70: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


Opening raw data file ../../data/mne_raw/sub0-day2-jhana-raw.fif...
    Range : 256 ... 385063 =      1.000 ...  1504.152 secs
Ready.
Reading 0 ... 384807  =      0.000 ...  1503.152 secs...


...

/opt/anaconda3/envs/hypyp/lib/python3.8/site-packages/neurokit2/ecg/ecg_plot.py:70: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


Opening raw data file ../../data/mne_raw/sub8-control-mindfulness-raw.fif...
    Range : 512 ... 386863 =      2.000 ...  1511.184 secs
Ready.
Reading 0 ... 386351  =      0.000 ...  1509.184 secs...


/opt/anaconda3/envs/hypyp/lib/python3.8/site-packages/neurokit2/ecg/ecg_plot.py:70: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


Opening raw data file ../../data/mne_raw/sub9-control-mindfulness-raw.fif...
    Range : 512 ... 402463 =      2.000 ...  1572.121 secs
Ready.
Reading 0 ... 401951  =      0.000 ...  1570.121 secs...


/opt/anaconda3/envs/hypyp/lib/python3.8/site-packages/neurokit2/ecg/ecg_plot.py:70: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


### REPORT

The only file that didn't work is this one: 
FAILURE: sub3-day4-jhana_mmn-raw.fif

# Read data

In [99]:
df_all = pd.read_excel('output.xlsx')
